# 1 : Import required libraries

In [1]:
import pandas as pd
import pulp
from pulp import *
import warnings

# Suppress unnecessary warnings for cleaner output
warnings.filterwarnings('ignore')


# 2 : Read the Data

In [2]:
data = pd.read_csv('Vegetarian_Ingredients_Nutrition.csv')
data.head()


,Foods,Price/Serving (INR),Serving Size,Calories,Protein (g),Total_Fat (g),Carbohydrates (g),Dietary_Fiber (g),Calcium (mg),Iron (mg),Vit_A (µg),Vit_C (mg)
0,Rice (Raw),5.0,100 g,130.0,2.7,0.3,28.0,0.4,10.0,0.2,0.0,0.0
1,Wheat Flour,4.0,100 g,364.0,12.0,1.5,76.0,2.5,15.0,1.2,0.0,0.0
2,Lentils (Dal),10.0,100 g,116.0,9.0,0.4,20.0,8.0,56.0,2.0,15.0,1.0
3,Spinach (Palak),4.0,100 g,23.0,2.9,0.4,3.6,2.2,99.0,2.7,469.0,28.0
4,Potato,3.0,100 g,77.0,2.0,0.1,17.0,2.2,10.0,0.8,10.0,19.7


# 3 : Define the main optimization function

#     Initialize the optimization problem

#     Solve the optimization problem and process results

#     Process and collect the optimization results

In [3]:
# Define solve_diet_optimization function (Part 1 - Setup and Data Preparation)
def solve_diet_optimization(data_path, print_details=True):
    """
    Solve a balanced diet optimization problem using linear programming.
    
    Parameters:
    -----------
    data_path : str
        Path to the CSV file containing nutritional information
    print_details : bool, optional
        Whether to print detailed optimization information (default is True)
    
    Returns:
    --------
    tuple
        A tuple containing:
        - List of selected foods with quantities
        - Total nutrition summary
        - Total cost of the diet
    """
    # Read and prepare data
    df = pd.read_csv(data_path)
    df = df.dropna()
    df['Foods'] = df['Foods'].str.strip()
    
    # Define food groups
    grains = ['Rice (Raw)', 'Wheat Flour']
    legumes = ['Lentils (Dal)', 'Chickpeas']
    vegetables = ['Spinach (Palak)', 'Potato', 'Tomato', 'Carrots']
    dairy = ['Milk']


    
    # Create optimization problem (Part 2 - Optimization Problem)
    prob = LpProblem('Diet_Optimization', LpMinimize)
    
    # Decision variables
    food_vars = LpVariable.dicts("Food", df['Foods'], lowBound=0, cat='Continuous')
    
    # Objective function: Minimize cost
    prob += lpSum([food_vars[food] * price for food, price in zip(df['Foods'], df['Price/Serving (INR)'])])
    
    # Nutritional constraints
    prob += lpSum([food_vars[food] * cal for food, cal in zip(df['Foods'], df['Calories'])]) >= 650  # Per Meal calories
    prob += lpSum([food_vars[food] * protein for food, protein in zip(df['Foods'], df['Protein (g)'])]) >= 15  # Protein
    prob += lpSum([food_vars[food] * carbs for food, carbs in zip(df['Foods'], df['Carbohydrates (g)'])]) >= 100  # Carbs
    prob += lpSum([food_vars[food] * fiber for food, fiber in zip(df['Foods'], df['Dietary_Fiber (g)'])]) >= 8  # Fiber
    prob += lpSum([food_vars[food] * fat for food, fat in zip(df['Foods'], df['Total_Fat (g)'])]) <= 10  # Fat
    prob += lpSum([food_vars[food] * calcium for food, calcium in zip(df['Foods'], df['Calcium (mg)'])]) >= 233  # Calcium
    prob += lpSum([food_vars[food] * iron for food, iron in zip(df['Foods'], df['Iron (mg)'])]) >= 6  # Iron
    prob += lpSum([food_vars[food] * vita for food, vita in zip(df['Foods'], df['Vit_A (µg)'])]) >= 260  # Vitamin A
    prob += lpSum([food_vars[food] * vitc for food, vitc in zip(df['Foods'], df['Vit_C (mg)'])]) >= 13  # Vitamin C
    
    # Food group constraints as per requirements
    # Exactly 2 servings from grains
    prob += lpSum([food_vars[food] for food in grains]) == 2
    
    #Exactly 1 servings from legumes
    prob += lpSum([food_vars[food] for food in legumes]) == 1
    
    # Exactly 2 servings from vegetables
    prob += lpSum([food_vars[food] for food in vegetables]) == 2
    
    # Exactly 1 serving from dairy
    prob += lpSum([food_vars[food] for food in dairy]) == 1
    
    # Individual food constraints
    for food in df['Foods']:
        prob += food_vars[food] <= 2  # Maximum 2 servings of any single food
    
    # Solve the problem
    prob.solve()
    
    # Check if the problem has been solved successfully
    if LpStatus[prob.status] != 'Optimal':
        print(f"Optimization failed with status: {LpStatus[prob.status]}")
        return None  # Return None if optimization fails
    
    if print_details:
        print(f"Status: {LpStatus[prob.status]}")


    
    # Process results (Part 3 - Result Processing)
    selected_foods = []
    total_nutrition = {
        'Calories': 0, 'Protein (g)': 0, 'Total_Fat (g)': 0,
        'Carbohydrates (g)': 0, 'Dietary_Fiber (g)': 0,
        'Calcium (mg)': 0, 'Iron (mg)': 0, 'Vit_A (µg)': 0, 'Vit_C (mg)': 0
    }
    
    for food in df['Foods']:
        value = food_vars[food].value()
        if value > 0.01:  # Only include foods with non-zero quantities
            food_data = df[df['Foods'] == food].iloc[0]
            cost = value * food_data['Price/Serving (INR)']
            
            # Determine food group
            group = 'Other'
            if food in grains: group = 'Grains'
            elif food in legumes: group = 'Legumes'
            elif food in vegetables: group = 'Vegetables'
            elif food in dairy: group = 'Dairy'
            
            selected_foods.append({
                'Food': food,
                'Quantity': value,
                'Cost': cost,
                'Food_Group': group
            })
            
            # Calculate nutrition totals
            for nutrient in total_nutrition.keys():
                if nutrient in food_data:
                    total_nutrition[nutrient] += value * food_data[nutrient]
    
    return selected_foods, total_nutrition, prob.objective.value()


# 4 : Define the reporting function

In [4]:
#  Reporting Function
def print_diet_report(foods, nutrition, cost):
    """Print a detailed report of the optimized diet plan."""
    print("\n" + "="*70)
    print("OPTIMIZED DAILY DIET PLAN")
    print("="*70)
    
    # SECTION 1: Food Items by Group
    print("\n1. RECOMMENDED DAILY SERVINGS")
    print("-"*50)
    
    # Group foods by category
    by_group = {}
    for item in foods:
        group = item['Food_Group']
        if group not in by_group:
            by_group[group] = []
        by_group[group].append(item)
    
    # Print foods in specific order
    for group in ['Grains', 'Legumes', 'Vegetables', 'Dairy']:
        if group in by_group:
            print(f"\n{group}:")
            group_total = sum(item['Quantity'] for item in by_group[group])
            for item in by_group[group]:
                print(f"  - {item['Food']}: {item['Quantity']:.2f} servings (₹{item['Cost']:.2f})")
            print(f"  Total {group}: {group_total:.2f} servings")
    
     # SECTION 2: Nutrition Summary
    print("\n2. NUTRITION SUMMARY")
    print("-"*50)
    print(f"Calories: {nutrition['Calories']:.1f}")
    print(f"Protein: {nutrition['Protein (g)']:.1f}g")
    print(f"Carbohydrates: {nutrition['Carbohydrates (g)']:.1f}g")
    print(f"Fat: {nutrition['Total_Fat (g)']:.1f}g")
    print(f"Fiber: {nutrition['Dietary_Fiber (g)']:.1f}g")
    print(f"Calcium: {nutrition['Calcium (mg)']:.1f}mg")
    print(f"Iron: {nutrition['Iron (mg)']:.1f}mg")
    print(f"Vitamin A: {nutrition['Vit_A (µg)']:.1f}µg")
    print(f"Vitamin C: {nutrition['Vit_C (mg)']:.1f}mg")
    
    # SECTION 3: Cost Analysis
    print("\n3. COST ANALYSIS")
    print("-"*50)
    print(f"Total Daily Cost: ₹{cost:.2f}")
    print(f"Average Cost per Serving: ₹{cost/sum(item['Quantity'] for item in foods):.2f}")
    
    print("\n" + "="*70)


# 5 : Main execution

In [5]:
# Main execution
if __name__ == "__main__":
    DATA_PATH = 'Vegetarian_Ingredients_Nutrition.csv'  # Path to your dataset
    foods, nutrition, cost = solve_diet_optimization(DATA_PATH)  # Solve the optimization problem
    
    if foods is not None:
        print_diet_report(foods, nutrition, cost)  # Print the results
    else:
        print("No feasible diet found.")


Status: Optimal

OPTIMIZED DAILY DIET PLAN

1. RECOMMENDED DAILY SERVINGS
--------------------------------------------------

Grains:
  - Rice (Raw): 1.23 servings (₹6.16)
  - Wheat Flour: 0.77 servings (₹3.07)
  Total Grains: 2.00 servings

Legumes:
  - Lentils (Dal): 1.00 servings (₹10.00)
  Total Legumes: 1.00 servings

Vegetables:
  - Spinach (Palak): 0.60 servings (₹2.38)
  - Potato: 1.40 servings (₹4.21)
  Total Vegetables: 2.00 servings

Dairy:
  - Milk: 1.00 servings (₹14.00)
  Total Dairy: 1.00 servings

2. NUTRITION SUMMARY
--------------------------------------------------
Calories: 801.5
Protein: 29.4g
Carbohydrates: 150.9g
Fat: 10.0g
Fiber: 14.8g
Calcium: 392.9mg
Iron: 6.0mg
Vitamin A: 428.5µg
Vitamin C: 45.3mg

3. COST ANALYSIS
--------------------------------------------------
Total Daily Cost: ₹39.83
Average Cost per Serving: ₹6.64



# ============================================================================================================================================

# Result visualization

In [6]:
# Import required libraries
import dash
import dash_bootstrap_components as dbc  # For styling components
import plotly.express as px  # For easy plotting
import plotly.graph_objects as go  # For more customizable plots
import pandas as pd
from dash import dcc, html  # Core Dash components

# Sample food data with groups, items, servings, and costs
data = {
    "Food Groups": ["Grains", "Grains", "Legumes", "Vegetables", "Vegetables", "Dairy"],
    "Item": ["Rice ", "Wheat Flour", "Lentils ", "Spinach ", "Potato", "Milk"],
    "Servings": [1.23, 0.77, 1.00, 0.60, 1.40, 1.00],
    "Cost (₹)": [6.16, 3.07, 10.00, 2.38, 4.21, 14.00]
}
df = pd.DataFrame(data)

# Nutritional information data
nutrition = {
    "Nutrient": ["Calories", "Protein", "Total Fat", "Carbohydrates", "Dietary Fiber", 
                 "Iron", "Vitamin A", "Vitamin C", "Calcium"],
    "Amount": [801.5, 29.4, 10.0, 150.9, 14.8, 6.0, 428.5, 45.3, 392.9]
}
df_nutrition = pd.DataFrame(nutrition)

# Create a donut chart for cost breakdown
fig_cost = px.pie(
    df, 
    values="Cost (₹)", 
    names="Item", 
    title="Cost Breakdown by Food Items",
    color_discrete_sequence=px.colors.sequential.RdBu,  # Color scheme
    hole=0.3  # Creates a donut chart effect
)

# Customize the cost chart appearance
fig_cost.update_layout(
    template="plotly_dark",
    paper_bgcolor="black",
    plot_bgcolor="black",
    font=dict(size=14, color="white")
)

# Create a bar chart for nutrition breakdown
fig_nutrition = go.Figure()

# Add each nutrient as a separate bar
for i, row in df_nutrition.iterrows():
    fig_nutrition.add_trace(
        go.Bar(
            x=[row["Nutrient"]],
            y=[row["Amount"]],
            text=[f'{row["Amount"]:.2f}'],  # Show values on bars
            textposition='outside',
            name=row["Nutrient"]
        )
    )

# Customize the nutrition chart appearance
fig_nutrition.update_layout(
    title="Nutrient Breakdown",
    template='plotly_dark',
    paper_bgcolor="black",
    plot_bgcolor="black"
)

# Initialize the Dash app with a dark theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# Define the app layout using Bootstrap components
app.layout = dbc.Container([
    # Title
    html.H1("🍽 Balanced Diet Optimization", className="text-center text-warning mb-4"),
    
    # Row containing both charts
    dbc.Row([
        dbc.Col([dcc.Graph(figure=fig_cost)], width=6),  # Cost breakdown chart
        dbc.Col([dcc.Graph(figure=fig_nutrition)], width=6)  # Nutrition breakdown chart
    ]),
    
    # Horizontal line separator
    html.Hr(),
    
    # Data table showing all food items
    dbc.Table.from_dataframe(
        df,
        striped=True,
        bordered=True,
        hover=True,
        dark=True
    ),
    
    # Summary statistics
    html.P("Total Daily Cost: ₹39.83", className="text-center text-light bg-dark p-2 rounded"),
    html.P("Average Cost per Serving: ₹6.64", className="text-center text-light bg-dark p-2 rounded")
], fluid=True, style={"backgroundColor": "black", "color": "white"})

#Run the model
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)